In [74]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import scipy
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from random import shuffle
import gc
import sklearn

import os
import codecs

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/task_data/
!ls

/content/drive/MyDrive/task_data
mfcc_13_2048.csv  train_data.csv  valid_data.csv  wavs


In [4]:
emd_file = '../amazon/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(emd_file, binary=True)
stops = set(stopwords.words("english"))
w2v_set = set(word2vec.vocab)

In [5]:
df = pd.read_csv('train_data.csv')
df.head()

,path,transcription,action,object,location
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none


In [6]:
onehot_encoder = OneHotEncoder(sparse=False, dtype='int32')
act_oh = onehot_encoder.fit_transform(np.array(df['action']).reshape(-1,1))
obj_oh = onehot_encoder.fit_transform(np.array(df['object']).reshape(-1,1))
loc_oh = onehot_encoder.fit_transform(np.array(df['location']).reshape(-1,1))
df['act_oh'] = list(act_oh)
df['obj_oh'] = list(obj_oh)
df['loc_oh'] = list(loc_oh)
df.head()

,path,transcription,action,object,location,act_oh,obj_oh,loc_oh
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen,"[1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]"
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none,"[0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]"
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none,"[0, 0, 1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]"
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom,"[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1]"
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none,"[0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 1, 0]"


In [59]:
df['class']=df['act_oh']
for i in range(len(df)):
  df['class'].iloc[i]= np.array(list(df['act_oh'].iloc[i])+list(df['obj_oh'].iloc[i])+list(df['loc_oh'].iloc[i])).reshape(1,-1).astype('float32')[0]
df.head()

,path,transcription,action,object,location,act_oh,obj_oh,loc_oh,class
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen,"[1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none,"[0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none,"[0, 0, 1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom,"[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none,"[0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 1, 0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
#pre pos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['transcription'])
print(len(tokenizer.word_index))

98


In [9]:
EMBEDDING_DIM= 300
nb_words = len(tokenizer.word_index)+1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
  if word in w2v_set:
    embedding_matrix[i] = word2vec.word_vec(word)
    # print(embedding_matrix[i].shape)
  else:
    embedding_matrix[i] = np.zeros(EMBEDDING_DIM,)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print(embedding_matrix.shape)

Null word embeddings: 6
(99, 300)


In [85]:
max(df['transcription'].apply(lambda x: len(x.split())))

10

In [11]:
data = tokenizer.texts_to_sequences(df['transcription'].values)
data = pad_sequences(data, maxlen=10)

In [64]:
X = data
Y = pd.DataFrame()

for i in range(23):
  Y[i+1]=df['class'].apply(lambda x: int(x[i]))

print(Y)

VALIDATION_SPLIT = 0.15
perm = np.random.permutation(len(X))
idx_train = perm[:int(len(X)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(X)*(1-VALIDATION_SPLIT)):]
print(len(idx_train), len(idx_val))

X_train = X[idx_train]
X_val = X[idx_val]
Y_train = Y.iloc[idx_train,:]
Y_val = Y.iloc[idx_val,:]

print(X_train.shape, X_val.shape)
print(Y_train.shape, Y_val.shape)

       1   2   3   4   5   6   7   8   9   ...  15  16  17  18  19  20  21  22  23
0       1   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   1   0
1       0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   0   1
2       0   0   1   0   0   0   1   0   0  ...   0   0   0   0   0   0   0   0   1
3       0   0   0   0   1   0   0   0   0  ...   0   0   0   0   0   0   0   0   0
4       0   0   1   0   0   0   0   0   0  ...   0   0   1   0   0   0   0   0   1
...    ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..
11561   0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   1   0
11562   0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   0   1
11563   0   1   0   0   0   0   0   0   0  ...   0   0   0   1   0   0   0   0   1
11564   0   0   0   0   1   0   0   0   0  ...   0   0   0   0   0   1   0   0   1
11565   0   0   0   1   0   0   0   0   0  ...   0   0   0   0   0   0   0   1   0

[11

In [20]:
#Model
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=10,
        trainable=False)

lstm_layer_1 = LSTM(20, return_sequences=False, dropout=0.15, recurrent_dropout=0.15)
#lstm_layer_2 = LSTM(200, return_sequences=False, dropout=0.15, recurrent_dropout=0.15)

sequence_input = Input(shape=(10,), dtype='int32')
x1 = embedding_layer(sequence_input)
x1 = lstm_layer_1(x1)
#x1 = lstm_layer_2(x1)
x1 = Dropout(0.10)(x1)
x1 = BatchNormalization()(x1)
x1 = Dense(50, activation='relu')(x1)
x1 = Dropout(0.10)(x1)
x1 = BatchNormalization()(x1)
preds = Dense(23, activation='sigmoid')(x1)

model = Model(inputs=sequence_input, outputs=preds)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 300)           29700     
                                                                 
 lstm (LSTM)                 (None, 20)                25680     
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 batch_normalization (BatchN  (None, 20)               80        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 50)                1050      
                                                             

In [82]:
# Y_train = tf.convert_to_tensor(Y_train)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

loss_fn = tf.keras.losses.MeanAbsoluteError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
train_acc_metric = tf.keras.losses.MeanAbsoluteError()
val_acc_metric = sklearn.metrics.f1_score

In [83]:
model.compile(optimizer=optimizer, loss=loss_fn, metrics=train_acc_metric)

In [84]:
model.fit(X_train, Y_train, batch_size=1024, epochs=10, validation_data=(X_val, Y_val), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 31s 248ms/step - loss: 0.0053 - mean_absolute_error: 0.0053 - val_loss: 0.0050 - val_mean_absolute_error: 0.0049
Epoch 2/10
10/10 [==============================] - 2s 206ms/step - loss: 0.0053 - mean_absolute_error: 0.0053 - val_loss: 0.0050 - val_mean_absolute_error: 0.0049
Epoch 3/10
10/10 [==============================] - 2s 210ms/step - loss: 0.0052 - mean_absolute_error: 0.0052 - val_loss: 0.0050 - val_mean_absolute_error: 0.0049
Epoch 4/10
10/10 [==============================] - 2s 209ms/step - loss: 0.0052 - mean_absolute_error: 0.0052 - val_loss: 0.0050 - val_mean_absolute_error: 0.0049
Epoch 5/10
10/10 [==============================] - 2s 207ms/step - loss: 0.0053 - mean_absolute_error: 0.0053 - val_loss: 0.0050 - val_mean_absolute_error: 0.0049
Epoch 6/10
10/10 [==============================] - 2s 211ms/step - loss: 0.0052 - mean_absolute_error: 0.0052 - val_loss: 0.0050 - val_mean_absolute_error: 0.0049
Epoch 7/10
10/1

In [ ]:
# Test Script
text = input()
df= pd.read_csv(text)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['transcription'])

max_len = max(df['transcription'].apply(lambda x: len(x.split())))
data = tokenizer.texts_to_sequences(df['transcription'].values)
data = pad_sequences(data, maxlen=max_len)

preds = model.predict(data)
ans = pd.DataFrame
